In [ ]:
from glob import glob
import pandas as pd

In [ ]:
THEMATIQUES={
    "accord_methode_penibilite":"Accords de méthode (pénibilité)",
"accord_methode_pse":"Accords de méthode (PSE)",
"amenagement_temps_travail":"Aménagement du temps de travail (modulation, annualisation, cycles)",
"autres":"Autre, précisez",
"autres_condition_travail":"Autres dispositions de conditions de travail (CHSCT, médecine du travail, politique générale de prévention)",
"autres_dispositions_duree":"Autres dispositions durée et aménagement du temps de travail ",
"autres_dispositions_egalite":"Autres dispositions Egalité professionnelle",
"autres_dispositions_emploi":"Autres dispositions emploi",
"calendrier_negociation":"Calendrier des négociations",
"classifications":"Classifications",
"commision_paritaire":"Commissions paritaires",
"cet":"Compte épargne temps",
"couverture_complementaire":"Couverture complémentaire santé - maladie",
"don_jour":"Dispositifs don de jour et jour de solidarité",
"distribution_actions_gratuites":"Distribution d'actions gratuites",
"droit_deconnexion":"Droit à la déconnexion et outils numériques",
"droit_syndical":"Droit syndical, IRP, expression des salariés",
"duree_collective_temps_travail":"Durée collective du temps de travail",
"egalite_salariale":"Egalité salariale F/H",
"election_pro":"Elections professionnelles, prorogations de mandat et vote électronique",
"evolution_prime":"Evolution des primes",
"evolution_salariale":"Evolution des salaires (augmentation, gel, diminution)",
"fin_conflit":"Fin de conflit",
"conges":"Fixation des congés (jours fériés, ponts, RTT)",
"forfait":"Forfaits (en heures, en jours)",
"formation_pro":"Formation professionnelle",
"gpec":"GPEC",
"heures_supp":"Heures supplémentaires (contingent, majoration)",
"indemnites":"Indemnités (dont kilométrique)",
"interessement":"Intéressement",
"mesure_age":"Mesures d'âge (seniors, contrat de génération...)",
"mobilite":"Mobilité (géographique, professionnelle - promotions)",
"diversite":"Non discrimination - Diversité",
"participation":"Participation",
"pee_peg":"PEE ou PEG",
"pei":"PEI",
"penibilite":"Pénibilité du travail (1% pénibilité, prévention, compensation/réparation)",
"perco_percoi":"PERCO et PERCOI",
"performance_collecte":"Performance collective (accord de compétitivité)",
"prevoyance_collective":"Prévoyance collective, autre que santé maladie",
"prime_partage_profit":"Prime de partage des profits",
"qvt":"QVT, conciliation vie personnelle/vie professionnelle",
"reprise_des_donnees":"Reprise des données",
"retraite_complementaire":"Retraite complémentaire - supplémentaire",
"rupture_conventionnelle_collective":"Rupture conventionnelle collective",
"stress_rps":"Stress, risques psycho-sociaux",
"supplement_participation":"Supplément de participation",
"supplement_interessement":"Supplément d'intéressement",
"systeme_prime":"Système de prime (autre qu'évolution)",
"système_de_remuneration":"Système de rémunération (autres qu'évolution)",
"teletravail":"Télétravail",
"travail_temps_partiel":"Travail à temps partiel",
"travail_nuit":"Travail de nuit",
"travail_dimanche":"Travail du dimanche",
"travailleurs_handicapes":"Travailleurs handicapés"}

In [ ]:
list_of_df=[]
for file in glob("results/*"):
    with open(file,"r") as f:
        lines=f.readlines()
    lines=[line for line in lines if ":" in line and line.split(" : ")[0] in set(THEMATIQUES.keys())]
    d = dict()
    for line in lines:
        cle, valeur = line.split(" : ")[0], line.split(" : ")[1]
        d[cle]=1 if valeur.lower().startswith("oui") else 0
    list_of_df.append(pd.DataFrame(d,index=[file.split("/")[1].split(".")[0]]))

In [ ]:
df_results=pd.concat(list_of_df)

In [ ]:
df_sample=pd.read_parquet("./10p_accords_publics_et_thematiques_240815_sample_of_1000.parquet")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

for (k,v) in THEMATIQUES.items():
    df=pd.DataFrame(df_sample[k].astype(int)).merge(df_results[k],how="inner",left_index=True,right_index=True,suffixes=["_expected","_predicted"])
    y_true, y_pred=df[f"{k}_expected"], df[f"{k}_predicted"]
    cm = confusion_matrix(y_true, y_pred)
    print(k)
    print(cm)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    report = classification_report(y_true, y_pred)
    
    print(f'Accuracy: {accuracy}')
    print(f'Precision (macro): {precision}')
    print(f'Recall (macro): {recall}')
    print(f'F1 Score (macro): {f1}')
    print("-"*10)
    print('Classification Report:')
    print(report)
